In [15]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [16]:
rating_matrix = pd.read_csv('rmat', sep=',')

In [17]:
rating_matrix.head(5)

,m1,m10,m100,m1000,m1002,m1003,m1004,m1005,m1006,m1007,...,m99,m990,m991,m992,m993,m994,m996,m997,m998,m999
u1,5.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u10,5.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u100,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u1000,5.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
u1001,4.0000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


The most popular movies will be defined as the top 10 movies when sorted by 1) number of reviews and 2) mean rating

In [20]:
# Step 1: Calculate the mean rating for each movie
movie_means = rating_matrix.mean(axis=0)

# Step 2: Calculate the number of ratings for each movie
movie_counts = rating_matrix.count(axis=0)

# Step 3: Create a DataFrame with mean ratings and counts
movie_stats = pd.DataFrame({
    'mean_rating': movie_means,
    'rating_count': movie_counts
})

# Step 4: Filter movies with average rating > 4.0
# filtered_movies = movie_stats[movie_stats['mean_rating'] > 4.25]

# Step 5: Sort the filtered movies by rating count in descending order
sorted_movies = movie_stats.sort_values(by=['rating_count', 'mean_rating'], ascending=False)
# Step 6: Select the top 10 movies
top_10_movies = sorted_movies.head(10)

# Display the top 10 movies with their average ratings and counts
print(top_10_movies)


       mean_rating  rating_count
m2858    4.3173862          3428
m260     4.4536944          2991
m1196    4.2929766          2990
m1210    4.0228928          2883
m480     3.7638473          2672
m2028    4.3373539          2653
m589     4.0585126          2649
m2571    4.3158301          2590
m1270    3.9903213          2583
m593     4.3518231          2578


In [21]:
from IPython.display import Image, display, HTML

# Step 1: Read the movies.dat file into a DataFrame with the correct encoding
movies_url = 'https://liangfgithub.github.io/MovieData/movies.dat?raw=true'
movies = pd.read_csv(
    movies_url,
    sep='::',
    header=None,
    engine='python',
    names=['movieID', 'title', 'genres'],
    encoding='latin1'  # Specifying the correct encoding
)

# Display the first few rows of the movies DataFrame
# print("Movies DataFrame Head:")
# print(movies.head())

# Assuming 'top_10_movies' is already defined from previous steps
# If not, ensure you have executed the steps to compute 'top_10_movies'

# Step 2: Extract numerical movie IDs from the column names
top_10_movies = top_10_movies.reset_index()
top_10_movies.rename(columns={'index': 'movieID_str'}, inplace=True)
top_10_movies['movieID'] = top_10_movies['movieID_str'].str.extract('m(\d+)').astype(int)

# Step 3: Merge the top 10 movies with the movies DataFrame to get titles
top_10_movies = top_10_movies.merge(movies[['movieID', 'title']], on='movieID', how='left')

# Display the merged DataFrame
# print("\nMerged Top 10 Movies with Titles:")
# print(top_10_movies[['movieID', 'title', 'mean_rating', 'rating_count']])

# Step 4: Construct image URLs for each movie
top_10_movies['image_url'] = 'https://liangfgithub.github.io/MovieImages/' + top_10_movies['movieID'].astype(str) + '.jpg'

# Display the DataFrame with image URLs
# print("\nTop 10 Movies with Image URLs:")
# print(top_10_movies[['movieID', 'title', 'image_url']])

# Step 5: Display titles and images in the Jupyter notebook
html = ''
for idx, row in top_10_movies.iterrows():
    html += f"""
    <div style="float:left; margin:20px; text-align:center">
        <img src="{row['image_url']}" style="width:150px; height:auto"><br>
        <strong>{row['title']}</strong>
    </div>
    """

display(HTML(html))



Display the pairwise similarity values from the 𝑆 matrix (you obtained at Step 2) for the following specified movies: “m1”, “m10”, “m100”, “m1510”, “m260”, “m3212”. Please round the results to 7 decimal places.

In [9]:
import pandas as pd
import numpy as np

# Step 1: Normalize the rating matrix by centering each row
user_means = rating_matrix.mean(axis=1)
rating_matrix_centered = rating_matrix.sub(user_means, axis=0)

# Define the function to compute similarity between two movies
def compute_similarity(movie_i, movie_j, rating_matrix_centered, min_common_users=3):
    # Get the ratings for both movies
    ratings_i = rating_matrix_centered[movie_i]
    ratings_j = rating_matrix_centered[movie_j]
    
    # Identify users who have rated both movies
    mask_i = ~ratings_i.isna()
    mask_j = ~ratings_j.isna()
    common_users = mask_i & mask_j
    
    num_common_users = common_users.sum()
    
    if num_common_users >= min_common_users:
        # Get the centered ratings for common users
        R_li = ratings_i[common_users]
        R_lj = ratings_j[common_users]
        
        # Compute numerator and denominators
        numerator = np.dot(R_li, R_lj)
        denominator = np.sqrt(np.sum(R_li ** 2)) * np.sqrt(np.sum(R_lj ** 2))
        
        if denominator == 0:
            similarity = np.nan
        else:
            cosine_similarity = numerator / denominator
            similarity = 0.5 + 0.5 * cosine_similarity  # Transform to [0, 1]
    else:
        # Not enough common users; similarity is NaN
        similarity = np.nan
    pd.set_option('display.float_format', '{:.7f}'.format)  # Set precision
    return similarity

# List of specified movies
specified_movies = ['m1', 'm10', 'm100', 'm1510', 'm260', 'm3212']

# Initialize an empty DataFrame to store similarities
similarity_matrix = pd.DataFrame(index=specified_movies, columns=specified_movies)

# Compute similarities for each pair of specified movies
for movie_i in specified_movies:
    for movie_j in specified_movies:
        if movie_i == movie_j:
            similarity = np.nan  # Similarity with itself is NaN
        else:
            similarity = compute_similarity(movie_i, movie_j, rating_matrix_centered)
        similarity_matrix.loc[movie_i, movie_j] = similarity

# Round the similarities to 7 decimal places
similarity_matrix = similarity_matrix.astype(float).round(7)

# Display the similarity matrix
print(similarity_matrix)


             m1       m10      m100  m1510      m260  m3212
m1          NaN 0.5121055 0.3919999    NaN 0.7411482    NaN
m10   0.5121055       NaN 0.5474583    NaN 0.5343338    NaN
m100  0.3919999 0.5474583       NaN    NaN 0.3296943    NaN
m1510       NaN       NaN       NaN    NaN       NaN    NaN
m260  0.7411482 0.5343338 0.3296943    NaN       NaN    NaN
m3212       NaN       NaN       NaN    NaN       NaN    NaN


In [11]:


def compute_similarity_matrix(rating_matrix_centered, min_common_users=3, top_n=30):
    movies = rating_matrix_centered.columns
    S = pd.DataFrame(index=movies, columns=movies, dtype=float)

    for idx_i, movie_i in enumerate(movies):
        # For progress monitoring
        if idx_i % 100 == 0:
            print(f"Processing movie {idx_i+1}/{len(movies)}")
        
        # Get users who rated movie_i
        ratings_i = rating_matrix_centered[movie_i]
        mask_i = ~ratings_i.isna()
        
        # Compute similarities with other movies
        similarities = {}

        for idx_j, movie_j in enumerate(movies):
            if movie_i == movie_j:
                continue

            # Skip if similarity already computed
            if pd.notna(S.loc[movie_i, movie_j]):
                continue

            # Get users who rated movie_j
            ratings_j = rating_matrix_centered[movie_j]
            mask_j = ~ratings_j.isna()
            
            # Find common users
            common_users = mask_i & mask_j
            num_common_users = common_users.sum()
            
            if num_common_users >= min_common_users:
                # Get centered ratings for common users
                R_li = ratings_i[common_users]
                R_lj = ratings_j[common_users]
                
                # Compute numerator and denominator
                numerator = np.dot(R_li, R_lj)
                denominator = np.sqrt(np.sum(R_li ** 2)) * np.sqrt(np.sum(R_lj ** 2))
                
                if denominator == 0:
                    similarity = np.nan
                else:
                    cosine_similarity = numerator / denominator
                    similarity = 0.5 + 0.5 * cosine_similarity  # Transform to [0,1]
            else:
                similarity = np.nan  # Not enough common users

            # Store the similarity symmetrically
            S.loc[movie_i, movie_j] = similarity
            S.loc[movie_j, movie_i] = similarity

        # Keep only top_n similarities per movie
        sims = S.loc[movie_i].dropna()
        if len(sims) > top_n:
            top_sims = sims.nlargest(top_n)
            S.loc[movie_i] = np.nan
            S.loc[movie_i, top_sims.index] = top_sims

    return S

# Compute and save the similarity matrix S
# Uncomment the following lines to compute S
# S = compute_similarity_matrix(rating_matrix_centered)
# S.to_csv('similarity_matrix_S.csv')



Processing movie 1/3706
Processing movie 101/3706
Processing movie 201/3706
Processing movie 301/3706
Processing movie 401/3706
Processing movie 501/3706
Processing movie 601/3706
Processing movie 701/3706
Processing movie 801/3706
Processing movie 901/3706
Processing movie 1001/3706
Processing movie 1101/3706
Processing movie 1201/3706
Processing movie 1301/3706
Processing movie 1401/3706
Processing movie 1501/3706
Processing movie 1601/3706
Processing movie 1701/3706
Processing movie 1801/3706
Processing movie 1901/3706
Processing movie 2001/3706
Processing movie 2101/3706
Processing movie 2201/3706
Processing movie 2301/3706
Processing movie 2401/3706
Processing movie 2501/3706
Processing movie 2601/3706
Processing movie 2701/3706
Processing movie 2801/3706
Processing movie 2901/3706
Processing movie 3001/3706
Processing movie 3101/3706
Processing movie 3201/3706
Processing movie 3301/3706
Processing movie 3401/3706
Processing movie 3501/3706
Processing movie 3601/3706
Processing mo

In [12]:
def myIBCF(newuser):
    import pandas as pd
    import numpy as np

    # Load the similarity matrix S
    S = pd.read_csv('similarity_matrix_S.csv', index_col=0)

    # Ensure the order of movies in 'newuser' matches the columns of S
    movies = S.columns.tolist()
    w = pd.Series(newuser, index=movies)

    # Initialize a dictionary to store predictions
    predictions = {}

    # For movies not rated by the user, compute the predicted rating
    for movie_i in movies:
        if pd.notna(w[movie_i]):
            continue  # Skip movies already rated by the user

        # Get the similarities S_ij where S_ij is not NA and w_j is not NA
        S_i = S.loc[movie_i]
        S_i = S_i[S_i.notna()]

        # Get movies that the user has rated
        rated_movies = w[w.notna()].index.tolist()
        # Intersection of movies similar to movie_i and movies rated by user
        common_movies = list(set(S_i.index) & set(rated_movies))

        if len(common_movies) == 0:
            prediction = np.nan  # Cannot make a prediction
        else:
            # Compute the numerator and denominator
            S_ij = S_i[common_movies]
            w_j = w[common_movies]

            numerator = np.dot(S_ij, w_j)
            denominator = S_ij.sum()

            if denominator == 0:
                prediction = np.nan
            else:
                prediction = numerator / denominator

        predictions[movie_i] = prediction

    # Convert predictions to a Series
    predictions_series = pd.Series(predictions)

    # Get top 10 recommendations
    top_recommendations = predictions_series.dropna().sort_values(ascending=False)

    if len(top_recommendations) >= 10:
        recommended_movies = top_recommendations.head(10).index.tolist()
    else:
        # Need to fill remaining recommendations with popular movies
        num_needed = 10 - len(top_recommendations)
        # Load or compute the popularity ranking
        try:
            popularity_ranking = pd.read_csv('movie_popularity_ranking.csv', index_col=0)
        except FileNotFoundError:
            # Compute the popularity ranking based on System I
            # Assuming 'rating_matrix' is available
            movie_means = rating_matrix.mean(axis=0)
            movie_counts = rating_matrix.count(axis=0)
            popularity_ranking = pd.DataFrame({
                'mean_rating': movie_means,
                'rating_count': movie_counts
            })
            popularity_ranking = popularity_ranking.sort_values(by=['rating_count', 'mean_rating'], ascending=False)
            popularity_ranking.to_csv('movie_popularity_ranking.csv')

        # Exclude movies already rated by user and already recommended
        movies_already_rated = w[w.notna()].index.tolist()
        movies_already_recommended = top_recommendations.index.tolist()
        movies_to_exclude = set(movies_already_rated + movies_already_recommended)

        # Get movies from popularity ranking excluding movies_to_exclude
        remaining_movies = popularity_ranking.index.difference(movies_to_exclude)

        # Get the top movies needed
        additional_movies = remaining_movies[:num_needed]

        recommended_movies = top_recommendations.index.tolist() + additional_movies.tolist()

    return recommended_movies


In [13]:
# Get the ratings of user "u1181"
user_id = 'u1181'
newuser = rating_matrix.loc[user_id]

# Call the myIBCF function
recommendations = myIBCF(newuser)

# Print the top 10 recommendations
print(f"Top 10 recommendations for user {user_id}:")
for idx, movie in enumerate(recommendations, 1):
    print(f"{idx}. {movie}")


Top 10 recommendations for user u1181:
1. m3732
2. m749
3. m3899
4. m2361
5. m1253
6. m1235
7. m337
8. m1734
9. m3752
10. m2793


In [14]:
# Create a new user vector with all NaNs
newuser = pd.Series(index=rating_matrix.columns, dtype=float)

# Set the ratings for "m1613" and "m1755"
newuser['m1613'] = 5
newuser['m1755'] = 4

# Call the myIBCF function
recommendations = myIBCF(newuser)

# Print the top 10 recommendations
print("Top 10 recommendations for the hypothetical user:")
for idx, movie in enumerate(recommendations, 1):
    print(f"{idx}. {movie}")


Top 10 recommendations for the hypothetical user:
1. m1017
2. m765
3. m74
4. m3269
5. m691
6. m2870
7. m3466
8. m340
9. m338
10. m3258


### 100 most popular

In [23]:
import pandas as pd

# Example: assuming ratings_matrix is already loaded as a pandas DataFrame

# Step 1: Calculate the number of reviews (non-NaN values) for each movie
number_of_reviews = rating_matrix.notna().sum()

# Step 2: Calculate the mean rating for each movie
mean_rating = rating_matrix.mean()

# Step 3: Combine into a single DataFrame and sort
popularity_df = pd.DataFrame({
    'movie_id': rating_matrix.columns,
    'number_of_reviews': number_of_reviews,
    'mean_rating': mean_rating
})

# Sort by number of reviews (descending), then by mean rating (descending)
popularity_df = popularity_df.sort_values(by=['number_of_reviews', 'mean_rating'], ascending=[False, False])

# Step 4: Select the top 100 movies
top_100_movies = popularity_df.head(100)['movie_id']

# Step 5: Create a subset of the ratings matrix with the top 100 movies
ratings_matrix_top_100 = rating_matrix[top_100_movies]

# Display the resulting subset
ratings_matrix_top_100.head()


,m2858,m260,m1196,m1210,m480,m2028,m589,m2571,m1270,m593,...,m750,m2699,m39,m21,m1393,m2804,m588,m2406,m1220,m733
u1,NaN,4.0000000,NaN,NaN,NaN,5.0000000,NaN,NaN,5.0000000,NaN,...,NaN,NaN,NaN,NaN,NaN,5.0000000,4.0000000,NaN,NaN,NaN
u10,3.0000000,5.0000000,5.0000000,4.0000000,4.0000000,NaN,4.0000000,5.0000000,4.0000000,NaN,...,4.0000000,NaN,NaN,NaN,NaN,5.0000000,4.0000000,NaN,4.0000000,NaN
u100,NaN,4.0000000,4.0000000,4.0000000,3.0000000,4.0000000,NaN,3.0000000,NaN,4.0000000,...,NaN,NaN,NaN,4.0000000,NaN,NaN,NaN,4.0000000,NaN,NaN
u1000,NaN,5.0000000,3.0000000,5.0000000,4.0000000,NaN,5.0000000,5.0000000,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,4.0000000,4.0000000,4.0000000,4.0000000,NaN
u1001,5.0000000,NaN,NaN,NaN,NaN,4.0000000,NaN,4.0000000,NaN,NaN,...,NaN,NaN,NaN,NaN,3.0000000,NaN,NaN,NaN,NaN,4.0000000


In [24]:
# Compute and save the similarity matrix S
# Uncomment the following lines to compute S
# Step 1: Normalize the rating matrix by centering each row
user_means = ratings_matrix_top_100.mean(axis=1)
rating_matrix_top_100_centered = ratings_matrix_top_100.sub(user_means, axis=0)
S = compute_similarity_matrix(rating_matrix_top_100_centered)
S.to_csv('similarity_matrix_S_top_100.csv')


Processing movie 1/100


In [25]:
ratings_matrix_top_100.to_csv('rmat_top_100.csv')